In [1]:
import pandas as pd 
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import Trainer,TrainingArguments, BertTokenizer, BertModel, BertPreTrainedModel,BertConfig
from torch.utils.data import Dataset, DataLoader
from torch import nn
import warnings 
warnings.filterwarnings('ignore')

/Users/liu_ryan/opt/anaconda3/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/liu_ryan/opt/anaconda3/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


# 基于Bert的中文文本分类任务实现(上) 

## 环境 

## Python                3.7.10
## torch                 1.9.0
## transformers          4.8.1

## 读取数据以及一些数据分析

In [4]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       12000 non-null  int64 
 1   level_1  12000 non-null  object
 2   level_2  12000 non-null  object
 3   level_3  12000 non-null  object
 4   level_4  12000 non-null  object
 5   content  11998 non-null  object
 6   label    12000 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 656.4+ KB


In [6]:
train.isnull().sum()

id         0
level_1    0
level_2    0
level_3    0
level_4    0
content    2
label      0
dtype: int64

In [7]:
train['content'] = train['content'].fillna('')

In [8]:
train.isnull().sum()

id         0
level_1    0
level_2    0
level_3    0
level_4    0
content    0
label      0
dtype: int64

In [9]:
test.isnull().sum()

id         0
level_1    0
level_2    0
level_3    0
level_4    0
content    4
dtype: int64

In [10]:
test['content'] = test['content'].fillna('')

In [11]:
train.head(5)

id            level_1  level_2           level_3  \
0   0  工业/危化品类（现场）—2016版  （二）电气安全  6、移动用电产品、电动工具及照明   
1   1  工业/危化品类（现场）—2016版  （一）消防检查            1、防火巡查   
2   2  工业/危化品类（现场）—2016版  （一）消防检查            2、防火检查   
3   3  工业/危化品类（现场）—2016版  （一）消防检查            1、防火巡查   
4   4  工业/危化品类（现场）—2016版  （一）消防检查            1、防火巡查   

                                             level_4  \
0  1、移动使用的用电产品和I类电动工具的绝缘线，必须采用三芯(单相)或四芯(三相)多股铜芯橡套软线。   
1                           3、消防设施、器材和消防安全标志是否在位、完整；   
2                           6、重点工种人员以及其他员工消防知识的掌握情况；   
3                           3、消防设施、器材和消防安全标志是否在位、完整；   
4                  4、常闭式防火门是否处于关闭状态，防火卷帘下是否堆放物品影响使用；   

                      content  label  
0  使用移动手动电动工具,外接线绝缘皮破损,应停止使用.      0  
1                          一般      1  
2                     消防知识要加强      0  
3             消防通道有货物摆放 清理不及时      0  
4                     防火门打开状态      0

In [12]:
## 文本长度的信息
train['text'] = train['content']+train['level_1']+train['level_2']+train['level_3']+train['level_4']

In [13]:
test['text'] = test['content']+test['level_1']+test['level_2']+test['level_3']+test['level_4']

In [14]:
train['text_len'] = train['text'].map(len)
train['text_len'].describe()
## 平均长度是82 最大长度360 那么我们后面的建模过程可以用
## max_seq_len 人为设成

count    12000.000000
mean        78.433250
std         22.658626
min         41.000000
25%         64.000000
50%         73.000000
75%         90.000000
max        356.000000
Name: text_len, dtype: float64

In [15]:
train['text'][0]

'使用移动手动电动工具,外接线绝缘皮破损,应停止使用.工业/危化品类（现场）—2016版（二）电气安全6、移动用电产品、电动工具及照明1、移动使用的用电产品和I类电动工具的绝缘线，必须采用三芯(单相)或四芯(三相)多股铜芯橡套软线。'

In [16]:
train['label'][0]

0

In [23]:
def fill_padding(data, max_len):
    '''
    补全padding的函数，减少代码的重复, 等会需要用到 
    '''
    if len(data) < max_len:
        pad_len = max_len - len(data)
        padding = [0 for _ in range(pad_len)]
        data = torch.tensor(data + padding)
    else:
        data = torch.tensor(data[:mapax_len])
    return data    

## 重点：如何构建Bert的输入数据格式

In [31]:
class InputDataset(Dataset):
    
    def __init__(self,data,tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self,):
        return len(self.data)
    
    def __getitem__(self,item):
        # item是索引 用来取数据
        # 添加了特殊token的文本
        text = str(self.data['text'][item])
        # label
        label = self.data['label'][item]
        label = torch.tensor(label,dtype = torch.long)
        
        ## 方法1: 简单的实现自动构建 
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True, # 句子开头和分隔加cls sep 【cls】+ text +【sep】
            max_length = self.max_len,  # 最大长度
            return_token_type_ids=True,  #分句ids 0000000
            pad_to_max_length = True,  # 补全或截断
            return_attention_mask=True, # 注意力编码 111111000
            return_tensors='pt'  # pytorch类型
        )
        
        # 方法2：手动构建 input_ids
        tokens = tokenizer.tokenize(text)
        tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
        tokens_ids = [101] + tokens_ids + [102]
        input_ids = fill_padding(tokens_ids,self.max_len)
        
        # attention_mask
        attention_mask = [1 for _ in range(len(tokens_ids))] #11111111
        attention_mask = fill_padding(attention_mask, self.max_len)
        
        return {
             "text": text,  #文本数据
             "input_ids": input_ids,  # 通过tokenizer做的编码 101 。。。。 102
             "attention_mask": attention_mask,  #注意力编码111111000000
             "token_type_ids": encoding['token_type_ids'].flatten(),  #分句编码11111000000
             "label": label  #label 0或1
    }   

In [32]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
train_dataset = InputDataset(train, tokenizer, 128)

In [33]:
data_loader = DataLoader(train_dataset, batch_size=4)

In [34]:
batch = next(iter(data_loader))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [35]:
print(batch)

{'text': ['使用移动手动电动工具,外接线绝缘皮破损,应停止使用.工业/危化品类（现场）—2016版（二）电气安全6、移动用电产品、电动工具及照明1、移动使用的用电产品和I类电动工具的绝缘线，必须采用三芯(单相)或四芯(三相)多股铜芯橡套软线。', '一般工业/危化品类（现场）—2016版（一）消防检查1、防火巡查3、消防设施、器材和消防安全标志是否在位、完整；', '消防知识要加强工业/危化品类（现场）—2016版（一）消防检查2、防火检查6、重点工种人员以及其他员工消防知识的掌握情况；', '消防通道有货物摆放 清理不及时工业/危化品类（现场）—2016版（一）消防检查1、防火巡查3、消防设施、器材和消防安全标志是否在位、完整；'], 'input_ids': tensor([[ 101,  886, 4500, 4919, 1220, 2797, 1220, 4510, 1220, 2339, 1072,  117,
         1912, 2970, 5296, 5318, 5357, 4649, 4788, 2938,  117, 2418,  977, 3632,
          886, 4500,  119, 2339,  689,  120, 1314, 1265, 1501, 5102, 8020, 4385,
         1767, 8021,  100, 8112, 4276, 8020,  753, 8021, 4510, 3698, 2128, 1059,
          127,  510, 4919, 1220, 4500, 4510,  772, 1501,  510, 4510, 1220, 2339,
         1072, 1350, 4212, 3209,  122,  510, 4919, 1220,  886, 4500, 4638, 4500,
         4510,  772, 1501, 1469,  100, 5102, 4510, 1220, 2339, 1072, 4638, 5318,
         5357, 5296, 8024, 2553, 7557, 7023, 4500,  676, 5708,  113, 1296, 4685,
          1

In [36]:
print(batch['input_ids'].shape) # batchsize * maxlen
print(batch['attention_mask'].shape)
print(batch['token_type_ids'].shape)
print(batch['label'].shape)

torch.Size([4, 128])
torch.Size([4, 128])
torch.Size([4, 128])
torch.Size([4])


In [3]:
class BertForSeq(BertPreTrainedModel):
    
    def __init__(self,config):
        super(BertForSeq, self).__init__(config)
        self.config = BertConfig(config)
        self.num_labels = 2
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config)
        self.classifier = nn.Linear(config.hidden_size, self.num_labels)
        
        self.init_weights()
    def __forward__(
        self,
        input_ids,
        attention_mask = None,
        token_type_ids = None,
        labels = None,
        return_dict = None
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.bert(
            input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids,
            return_dict = return_dict

        )
        
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"
                    
            loss_fct = nn.CrossEntoryLoss ()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))  #二分类任务 这里的参数要做view
                    
        
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )    

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [1]:
tokenizer = BertTokenizer.from_pretrained(model_path)
model_path = 'bert-base-chinese'
def train_and_eval(max_len, batch_size, EPOCHS):
    model = BertForSeq.from_pretrained(model_path)
    train_dataset = InputDataset(train,tokenizer,max_len)
    train_dataloader = DataLoader(train_dataset, batch_size)
    test_dataset = InputDataset(test, tokenizer, max_len)
    test_dataloader = DataLoader(test_dataset,batch_size)
    # 优化器 为什么用AdamW
    optimizer = AdamW(model.parameters(), ler=2e-5)
    # 总步数
    total_steps = len(train_dataset) * EPOCHS
    # 调度器
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=total_steps
    )
    
    training_stats = []
    total_t0 = time.time()
    
    for epoch in range(EPOCHS):
        print('')
        print('==============Epoch{:}/{:}=============='.format(epoch+1, EPOCHS))
        print('Training')
        t0=time.time()
        total_train_loss=0
        model.to(device)
        model.train()
        
        for step, batch in enumerate(train_dataloader):
            if step % 20 == 0 and not step == 0:
                elasped = format_time(time.time()-t0)
                print('Batch {} of {},Elasped{}'.format(step,len(train_dataloader), elasped))
                
            # embedding 
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_batch'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)
                
            #梯度清零
            model.zero_grad()
            
            #输出和预测
            outputs = model(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                           labels = labels)
            
            loss = outputs.loss
            total_train_loss += loss.item()  # 不能直接相加
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            scheduler.step()
            
        avg_train_loss = total_train_loss / len(train_dataloader)
        training_time = 
            

SyntaxError: unexpected EOF while parsing (<ipython-input-1-3383a27cd080>, line 33)